In [1]:
# before binary (300, 300, 3)
# binaryMask shape : (300, 300)
# preprocess shape : (450, 450, 3)

In [2]:
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import copy
import cv2
import os
from imutils.video import FileVideoStream
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
import imutils
from tensorflow.keras.preprocessing.image import img_to_array

dataColor = (0,255,0)
font = cv2.FONT_HERSHEY_SIMPLEX
fx, fy, fh = 10, 50, 45
takingData = 0
className = 'NONE'
count = 0
showMask = 0


classes = 'NONE ONE TWO THREE FOUR FIVE'.split()


def initClass(name):
    global className, count
    className = name
    os.system('mkdir -p data/%s' % name)
    count = len(os.listdir('data/%s' % name))


def binaryMask(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (7,7), 3)
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    ret, new = cv2.threshold(img, 25, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return new


def preprocess(image, width, height):
    # grab the dimensions of the image, then initialize
    # the padding values
    (h, w) = image.shape[:2]

    # if the width is greater than the height then resize along
    # the width
    if w > h:
        image = imutils.resize(image, width=width)

    # otherwise, the height is greater than the width so resize
    # along the height
    else:
        image = imutils.resize(image, height=height)

    # determine the padding values for the width and height to
    # obtain the target dimensions
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)

    # pad the image then apply one more resizing to handle any
    # rounding issues
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))

    # return the pre-processed image
    return image

In [3]:
# cam = FileVideoStream("IMG_4009.MOV").start()
# frame = cap.read()
# vid_writer = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))

In [4]:
initClass('NONE')

global font, size, fx, fy, fh
global takingData, dataColor
global className, count
global showMask

model = load_model('model_6cat.h5')

x0, y0, width = 50,450,350#200, 220, 300
#x0, y0, width = 550, 500, 300

#cam = cv2.VideoCapture(0)
#cam = FileVideoStream("IMG_4009_1.mp4").start()
cam = cv2.VideoCapture('IMG_4009_1.mp4')
grabb,frame = cam.read()
vid_writer = cv2.VideoWriter('output1.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))
cv2.namedWindow('Original', cv2.WINDOW_NORMAL)

while True:
    # Get camera frame
    ret,frame = cam.read()
    if not ret:
        print("end")
        break;
    if frame is not None:

        #print(frame.shape)
        frame = cv2.rotate(frame,cv2.ROTATE_90_CLOCKWISE)
        #frame = cv2.flip(frame, 1) # mirror
        #frame = cv2.resize(frame, (300,300))
        window = copy.deepcopy(frame)

        cv2.rectangle(window, (x0,y0), (x0+width-1,y0+width-1), dataColor, 12)
        #cv2.rectangle(window, (x0,y0), (x0+400-1,y0+400-1), dataColor, 12)

        # draw text
        if takingData:
            dataColor = (0,250,0)
            #cv2.putText(window, 'Data Taking: ON', (fx,fy), font, 1.2, dataColor, 2, 1)
        else:
            dataColor = (0,0,250)
            #cv2.putText(window, 'Data Taking: OFF', (fx,fy), font, 1.2, dataColor, 2, 1)
        #cv2.putText(window, 'Class Name: %s (%d)' % (className, count), (fx,fy+fh), font, 1.0, (245,210,65), 2, 1)

        # get region of interest
        #roi = frame[y0:y0+width,x0:x0+width]
        #roi = binaryMask(roi)

        ############### CUSTOM CODE #################
        #roi = frame[y0:y0+500,x0:x0+500]
        roi = frame[y0:y0+width,x0:x0+width]
        roi = preprocess(roi,300,300)
        #print("after process",roi.shape)
        roi = binaryMask(roi)

        #print("after binaryMask shape : {}".format(roi.shape))

        #roi1 = frame[y0:y0+450,x0:x0+450]

        #roi = img_to_array(roi)
        #print("preprocess shape : {}".format(roi1.shape))
        #cv2.imshow("roi processed",roi)
        #print(roi.shape)       
        ############### END

        # apply processed roi in frame
        if showMask:            
            window[y0:y0+width,x0:x0+width] = cv2.cvtColor(roi, cv2.COLOR_GRAY2BGR)

        # take data or apply predictions on ROI
        if takingData:
             cv2.imwrite('data/{0}/{0}_{1}.png'.format(className, count), roi)
             count += 1
        else:
            img = np.float32(roi)/255.
            #img = np.array(roi, dtype="float") / 255.0
            img = np.expand_dims(img, axis=0)
            img = np.expand_dims(img, axis=-1)
            pred = classes[np.argmax(model.predict(img)[0])]
            #cv2.putText(window, 'Prediction: %s' % (pred), (fx,fy+2*fh), font, 1.0, (245,210,65), 2, 1)
            # use below for demoing purposes
            cv2.putText(window, 'Prediction: %s' % (pred), (x0,y0-25), font, 1.0, (255,0,0), 2, 1)
            #cv2.imshow('process roi', roi)
            #cv2.putText(roi, 'Prediction: %s' % (pred), (roi[roi.shape[1]/2.0,roi.shape[2]/2.0]), font, 1.0, (245,210,65), 2, 1)

        # show the window
        cv2.imshow('Original', window)
        
        #cv2.imshow('ROI', window)
        # Keyboard inputs
        key = cv2.waitKey(10) & 0xff
        
        # use q key to close the program
        if key == ord('q'):
            break
        
        # Toggle data taking
        elif key == ord('s'):            
            takingData = not takingData
        elif key == ord('b'):
            showMask = not showMask
        

        # Toggle class
        elif key == ord('0'):  initClass('NONE')
        elif key == ord('`'):  initClass('NONE') # because 0 is on other side of keyboard
        elif key == ord('1'):  initClass('ONE')
        elif key == ord('2'):  initClass('TWO')
        elif key == ord('3'):  initClass('THREE')
        elif key == ord('4'):  initClass('FOUR')
        elif key == ord('5'):  initClass('FIVE')

        # adjust the size of window
        #elif key == ord('z'):
        #    width = width - 5
        #elif key == ord('a'):
        #    width = width + 5

        # adjust the position of window
        elif key == ord('i'):
            y0 = max((y0 - 5, 0))
        elif key == ord('k'):
            y0 = min((y0 + 5, window.shape[0]-width))
        elif key == ord('j'):
            x0 = max((x0 - 5, 0))
        elif key == ord('l'):
            x0 = min((x0 + 5, window.shape[1]-width))
        vid_writer.write(window)

vid_writer.release()
cam.release()
cv2.destroyAllWindows()

end


In [5]:
# # if __name__ == '__main__':
# initClass('NONE')
# main()
# #147,894

In [6]:
# vid_writer.release()
# cv2.destroyAllWindows()
# cam.release()

In [7]:
# import cv2
# img = cv2.imread("test.JPG")
# scale_percent = 60 # percent of original size
# width = int(img.shape[1] * scale_percent / 100)
# height = int(img.shape[0] * scale_percent / 100)
# dim = (300, 300)
# # resize image
# img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
# cv2.imshow("test",img)
# cv2.waitKey(0)